In [2]:
import sqlite3
import numpy as np
import pandas as pd


In [3]:
path = './db/課題.sqlite'
con = sqlite3.connect(path)
cur = con.cursor()
sql_select_weather = "SELECT * FROM weather"
cur.execute(sql_select_weather)
data = cur.fetchall()
con.close()


In [4]:
df = pd.DataFrame(data, columns=["現地気圧", "海面気圧", "降水合計", "1時間降水", "10分降水", "平均気温", "最高気温", "最低気温",
    "平均湿度", "最小湿度", "平均風速", "最大風速", "風向", "最大瞬間風速", "最大瞬間風向", "日照時間", "降雪", "積雪", "昼", "夜", "月", "日"])
df

,現地気圧,海面気圧,降水合計,1時間降水,10分降水,平均気温,最高気温,最低気温,平均湿度,最小湿度,...,風向,最大瞬間風速,最大瞬間風向,日照時間,降雪,積雪,昼,夜,月,日
0,1013.5,1018.8,nan,nan,nan,7.4,12.1,2.3,59.0,39.0,...,北,11.2,北,9.5,nan,nan,晴,晴,1.0,1.0
1,1013.4,1018.6,0.0,0.0,0.0,6.9,11.1,4.3,51.0,30.0,...,北,9.6,北,6.3,nan,nan,晴後曇一時雨,晴時々曇,1.0,2.0
2,1012.8,1018.1,nan,nan,nan,6.5,10.9,3.0,45.0,36.0,...,北,9.6,北,9.4,nan,nan,晴,晴,1.0,3.0
3,1012.8,1018.1,nan,nan,nan,5.9,10.5,2.1,48.0,35.0,...,北,12.8,北,7.9,nan,nan,晴,晴,1.0,4.0
4,1016.4,1021.7,nan,nan,nan,6.2,11.0,2.9,36.0,20.0,...,北,9.7,北,8.9,nan,nan,晴,晴,1.0,5.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
360,1020.2,1025.4,nan,nan,nan,8.8,13.3,5.2,52.0,41.0,...,北,11.9,北,6.1,nan,nan,曇時々晴,曇時々晴,12.0,27.0
361,1020.1,1025.4,nan,nan,nan,8.1,11.4,6.4,53.0,40.0,...,北,8.6,北,4.5,nan,nan,晴時々曇,晴,12.0,28.0
362,1016.0,1021.2,nan,nan,nan,8.8,13.4,4.1,59.0,44.0,...,北北西,5.6,北,9.3,nan,nan,晴,晴,12.0,29.0
363,1014.9,1020.1,nan,nan,nan,9.8,14.6,5.9,60.0,38.0,...,北,7.6,北,8.3,nan,nan,晴,曇一時晴後一時雨,12.0,30.0
